# This notebook translates the log-who-is-home.csv to who-is-home-munged.csv
This allows for later analysis in "combined.xlsx"

In [2]:
import pandas as pd
from pandas import Timestamp
import matplotlib
import pytz
from datetime import datetime
from matplotlib import pyplot as plt
import numpy as np
import csv
import codecs
from pprint import pprint
from collections import OrderedDict

In [3]:
filename = r"raw-data\log-who-is-home.csv"
output_filename = r"raw-data\who-is-home-munged.csv"
columns = ['date', 'who', 'last_seen']

In [5]:
# read and munge input
lines = []

with open(filename, 'r') as f:
    reader = csv.reader(x.replace('\0', '') for x in f)
    lines = list(reader)
    
print(f'{len(lines)} lines read')
print(lines[:1])
# 1550978973 -> 2019-02-24 03:29:33 # note that this is UTC, and needs adjusting for TZ

469186 lines read
[['1550978973', 'Leland OP5t', '1550978917.777071']]


In [6]:
# convert to datetime, month, day, and hour, with correct TZ
# 2019-02-24 03:29:33   ->
# 2019-02-23 22:29:33-05:00
eastern = pytz.timezone('US/Eastern')
for line in lines:
    dt = datetime.utcfromtimestamp(int(line[0]))
    loc_dt = dt.replace(tzinfo=pytz.utc).astimezone(eastern)
    line.append(loc_dt.month)
    line.append(loc_dt.day)
    line.append(loc_dt.hour)
    loc_dt = loc_dt.replace(minute=0, second=0)  # remove info we don't need, so it can be used as a key in a dict
    loc_dt = loc_dt.replace(tzinfo=None)
    line.insert(0, loc_dt)
    #line.insert(0, f"2019-{loc_dt.month}-{loc_dt.day} {loc_dt.hour}:00:00")
print(lines[:1])


[[datetime.datetime(2019, 2, 23, 22, 0), '1550978973', 'Leland OP5t', '1550978917.777071', 2, 23, 22]]


In [7]:
# produce counts by bin, by person
counts_by_who = {}
# mapping for combining phones into one person
mapping = {
    'Leland OP5t': 'Leland',
    'Lucas iPhone': 'Lucas',
    'Greg iPhone': 'Greg',
    'Greg iPhone2': 'Greg',
    'Max iPhone': '4',
    'Devon GalaxyNote9': '4',
    'Devon Unk': '4'
}

all_bins = [] # needed for sorting/ordering times, later

for line in lines:
    time = line[0]
    all_bins.append(time)
    who = mapping.get(line[2], 'unk')
    if not counts_by_who.get(who,False):
        counts_by_who[who] = {}
    if not counts_by_who[who].get(time):
        counts_by_who[who][time] = 0
    counts_by_who[who][time] += 1

print(counts_by_who.keys())

dict_keys(['Leland', 'Lucas', 'Greg', '4'])


In [8]:
# reformat each by time-bins
print(all_bins[:10])
print('...')
print(all_bins[-10:])
all_bins.sort()
o = OrderedDict(zip(all_bins,[[] for i in range(len(all_bins))])) 

print(all_bins[:10])
print('...')
print(all_bins[-10:])


[datetime.datetime(2019, 2, 23, 22, 0), datetime.datetime(2019, 2, 23, 22, 0), datetime.datetime(2019, 2, 23, 22, 0), datetime.datetime(2019, 2, 23, 22, 0), datetime.datetime(2019, 2, 23, 22, 0), datetime.datetime(2019, 2, 23, 22, 0), datetime.datetime(2019, 2, 23, 22, 0), datetime.datetime(2019, 2, 23, 22, 0), datetime.datetime(2019, 2, 23, 22, 0), datetime.datetime(2019, 2, 23, 22, 0)]
...
[datetime.datetime(2019, 11, 30, 12, 0), datetime.datetime(2019, 11, 30, 12, 0), datetime.datetime(2019, 11, 30, 12, 0), datetime.datetime(2019, 11, 30, 12, 0), datetime.datetime(2019, 11, 30, 12, 0), datetime.datetime(2019, 11, 30, 12, 0), datetime.datetime(2019, 11, 30, 12, 0), datetime.datetime(2019, 11, 30, 12, 0), datetime.datetime(2019, 11, 30, 12, 0), datetime.datetime(2019, 11, 30, 12, 0)]
[datetime.datetime(2019, 2, 23, 22, 0), datetime.datetime(2019, 2, 23, 22, 0), datetime.datetime(2019, 2, 23, 22, 0), datetime.datetime(2019, 2, 23, 22, 0), datetime.datetime(2019, 2, 23, 22, 0), datetime

In [9]:
# fill the ordereddict
# counts_by_who
# counts_by_who(['Leland', 'Lucas', 'Greg', '4'])
for who, d in counts_by_who.items():
    for k,v in d.items():
        if o[k] == []:
            o[k] = {}
        o[k][who] = v

print(o)

OrderedDict([(datetime.datetime(2019, 2, 23, 22, 0), {'Leland': 28}), (datetime.datetime(2019, 2, 23, 23, 0), {'Leland': 31}), (datetime.datetime(2019, 2, 24, 16, 0), {'Leland': 44, 'Lucas': 46, 'Greg': 26}), (datetime.datetime(2019, 2, 24, 17, 0), {'Leland': 31, 'Lucas': 52, 'Greg': 18}), (datetime.datetime(2019, 2, 24, 18, 0), {'Lucas': 52, 'Greg': 9}), (datetime.datetime(2019, 2, 24, 19, 0), {'Lucas': 51, 'Greg': 52}), (datetime.datetime(2019, 2, 24, 20, 0), {'Lucas': 55, 'Greg': 53}), (datetime.datetime(2019, 2, 24, 21, 0), {'Lucas': 57, 'Greg': 51}), (datetime.datetime(2019, 2, 24, 22, 0), {'Lucas': 56, 'Greg': 50}), (datetime.datetime(2019, 2, 24, 23, 0), {'Lucas': 56, 'Greg': 52}), (datetime.datetime(2019, 2, 25, 0, 0), {'Leland': 52, 'Lucas': 56, 'Greg': 49}), (datetime.datetime(2019, 2, 25, 1, 0), {'Leland': 53, 'Lucas': 56, 'Greg': 50}), (datetime.datetime(2019, 2, 25, 2, 0), {'Leland': 54, 'Lucas': 57, 'Greg': 46}), (datetime.datetime(2019, 2, 25, 3, 0), {'Leland': 52, 'Luca

In [10]:
# throw into new CSV file
headers = ['time', 'Leland', 'Greg', 'Lucas', '4']

with open(output_filename, 'w', newline='') as csvfile:
    csvw = csv.writer(csvfile, delimiter=',')
    csvw.writerow(headers)
    for k,v in o.items():
        l = [k, v.get('Leland', 0), v.get('Greg', 0), v.get('Lucas', 0), v.get('4', 0)]
        csvw.writerow(l)